In [28]:
import json
import scipy.integrate as integ
from math import sqrt

In [58]:
class Seat:
    def __init__(self,ID,x,y,theta):
        self.x=x
        self.y=y
        self.theta=theta
        self.ID=ID
    def __compute_view_value__(self,ang,seat):
        eng = 0
        dist = sqrt(abs(self.x-seat.x)**2+abs(self.y-seat.y)**2)
        deltang = 1-abs(180-abs(self.theta-seat.theta))/180
        return dist*deltang

    def gradePosition(self,Seats):
        score = 0
        score = sum(self.__compute_view_value__(0,seat) for seat in Seats if seat.ID!=self.ID)
        return score

In [59]:
#Pull in data for a particular table
datasets = json.loads(open("table_data.json",'r').read())
table = 'picnic'
N = len(datasets[table].keys())
seats = {Seat(int(k),v[0],v[1],v[2]) for k,v in datasets[table].items()}

In [60]:
for seat in seats: (seat.gradePosition(seats))

In [61]:
datasets[table]

{'0': [0, 0, 180], '1': [2.5, 0, 180], '2': [0, 3, 0], '3': [2.5, 3, 0]}

In [62]:
for seat in seats: print(seat.gradePosition(seats))

6.905124837953327
6.905124837953327
6.905124837953327
6.905124837953327
